In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [2]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [3]:
def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [4]:
def cross_entropy_error(y, t):
    if y.dim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

# y.dim 으로 분기를 나눠주는 이유
# 1개의 데이터만 받아 오차값을 산정해야하는 경우를 처리해주기 위해서
# 함수를 더 범용적으로 사용하기 위함이다.

In [5]:
def numerical_diff(f, x):
    h = 1e-4 
    return (f(x+h) - f(x-h)) / (2*h)

# h를 0.0001로 해놓은 이유
# 10^(-4) 정도의 값을 이용하면 좋을 결괄르 얻는다고 알려져 있음

# 미분을 구하는 식인데 (f(x+h) - f(x)) / h 를 사용하지 않고 위의 식을 사용하는 이유
# 수치 미분은 근사치를 이용하여 사용하는 방식인데, 수치미분의 오차를 줄이기 위해서 중심 차분을 사용한다.

In [6]:
def function_2(x):
    return x[0]**2 + x[1]**2

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad

# 기울기가 카리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향

In [7]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [8]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num = 100)

array([-6.11110793e-10,  8.14814391e-10])

In [9]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [13]:
net = simpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

print(np.argmax(p))

t = np.array([0, 0, 1])
net.loss(x, t)

[[ 0.25566101  1.65791701 -0.30453892]
 [-1.16792913 -2.14112748 -0.71789921]]
[-0.89773961 -0.93226453 -0.82883264]
2


AttributeError: 'numpy.ndarray' object has no attribute 'dim'